<a href="https://colab.research.google.com/github/kumar987789/daily-updates/blob/main/nani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
movies
movieId,movieName,year
1,Parmanu,2018
2,URI,2019
3,rrr,2022
4,Bahubali,2015
5,tanhaji,2020
reviews
movieId,movieInfo,reviews
1,,435
2,,492
3,,444
4,,880
5,,722
6,,888
7,,220
8,,350
9,,619
10,,226

In [1]:
%%writefile movies.csv
movieId,movieName,year
1,Parmanu,2018
2,URI,2019
3,rrr,2022
4,Bahubali,2015
5,tanhaji,2020

Writing movies.csv


In [3]:
%%writefile reviews.csv
movieId,movieInfo,reviews
1,,435
2,,492
3,,444
4,,880
5,,722
6,,888
7,,220
8,,350
9,,619
10,,226

Writing reviews.csv


In [6]:
!pip install pyspark


In [56]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat,col,lit,min,max,expr,count

In [9]:
spark=SparkSession.builder.appName('abc').getOrCreate()

In [10]:
df=spark.read.option('header','True').csv('movies.csv')
df1=spark.read.option('header','True').csv('reviews.csv')

In [ ]:
#  Clean reviews dataframe on the basis of movies data available. Hint - try left semi join, inner join, right join for joins
# 2.  Fill movieInfo Column by concating movieName & year column (output should be dataframe).  Hint - try expr method
# 3. Find movie with max reviews (output should be string)
# 4. Find total reviews (output should be int)

In [54]:
#1)Clean reviews dataframe on the basis of movies data available. Hint - try left semi join, inner join, right join for joins

df.join(df1,df.movieId==df1.movieId,'leftsemi').show()


+-------+---------+----+
|movieId|movieName|year|
+-------+---------+----+
|      1|  Parmanu|2018|
|      2|      URI|2019|
|      3|      rrr|2022|
|      4| Bahubali|2015|
|      5|  tanhaji|2020|
+-------+---------+----+



In [39]:
x=df.join(df1,df.movieId==df1.movieId,'left')
#x.withColumn('movieInfo',concat(col('movieName'),lit(''),col('year'))).show()
x.withColumn('movieInfo',expr('concat(movieName," ",year)')).show()


+-------+---------+----+-------+-------------+-------+
|movieId|movieName|year|movieId|    movieInfo|reviews|
+-------+---------+----+-------+-------------+-------+
|      1|  Parmanu|2018|      1| Parmanu 2018|    435|
|      2|      URI|2019|      2|     URI 2019|    492|
|      3|      rrr|2022|      3|     rrr 2022|    444|
|      4| Bahubali|2015|      4|Bahubali 2015|    880|
|      5|  tanhaji|2020|      5| tanhaji 2020|    722|
+-------+---------+----+-------+-------------+-------+



In [50]:
#x.groupBy('movieName').agg(max('reviews')).show(1)
x.select('movieName').orderBy('reviews',ascending=False).collect()[0][0]

'Bahubali'

In [57]:
df1.select('reviews').agg({'reviews':'sum'}).withColumnRenamed('sum(reviews)','total_reviews').collect()[0][0]

5276.0

In [58]:
df1.select('reviews').agg({'reviews':'count'}).withColumnRenamed('sum(reviews)','total_reviews').collect()[0][0]

10

In [ ]:
moviesDF
movieId,movieName,year,fansReview,criticsReview
1,Parmanu,2018,200,100
2,URI,2019,300,200
3,rrr,2022,300,200
4,Bahubali,2015,500,400
5,tanhaji,2020,600,500
genreDF
movieId,genre
1,tech
2,history
2,action
4,action
3,action
1,science
4,fiction
5,history
5,action
4,mass

In [59]:
%%writefile moviesdf.csv
movieId,movieName,year,fansReview,criticsReview
1,Parmanu,2018,200,100
2,URI,2019,300,200
3,rrr,2022,300,200
4,Bahubali,2015,500,400
5,tanhaji,2020,600,500

Writing moviesdf.csv


In [64]:
%%writefile genredf.csv
movieId,genre
1,tech
2,history
2,action
4,action
3,action
1,science
4,fiction
5,history
5,action
4,mass

Overwriting genredf.csv


In [61]:
spark=SparkSession.builder.appName('def').getOrCreate()

In [74]:
k=spark.read.option('header','True').csv('moviesdf.csv')
k1=spark.read.option('header','True').csv('genredf.csv')

In [68]:

#Find top movies by fansReview to criticsReview ratio. Hint - Add a column for same
#Order moviesDF by year to get latest movie
#Join dataframes on movieID & find most popular genr

In [85]:
k.withColumn('ratio',col('fansReview')/col('criticsReview'))
k.orderBy('year',ascending=False).show(1)

+-------+---------+----+----------+-------------+
|movieId|movieName|year|fansReview|criticsReview|
+-------+---------+----+----------+-------------+
|      3|      rrr|2022|       300|          200|
+-------+---------+----+----------+-------------+
only showing top 1 row



In [81]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat,col,lit,min,max,expr,count,mode

In [102]:
x=k.join(k1,'movieId')
#x.select(mode('genre')).show()
y=x.groupby('genre').agg({'genre':'count'}).orderBy('count(genre)',ascending=False)

In [103]:
print(y.rdd.first()[0])

action


In [106]:
y.rdd.flatMap(lambda x:[x[0]]).collect()

['action', 'history', 'mass', 'fiction', 'science', 'tech']